# Реализация LU- и LUP- алгоритмов для решения СЛАУ
## Белов Никита Дмитриевич НПМбд-01-19

Импортируем библиотеку numpy.

In [1]:
import numpy as np

## LU-алгоритм

Преобразуем матрицу А в L и U матрицы. Cкопируем А в U, L сделаем единичной при помощи встроенной функции np.eye(). Используем переменную f как вспомогательную.

In [73]:
def a_to_lu(a):
    rows = a.shape[0]
    u = a.copy()
    l = np.eye(rows)
    for i in range(rows):
        f = u[i+1:, i] / u[i, i]
        l[i+1:, i] = f
        u[i+1:] -= f[:, np.newaxis] * u[i]
    return l, u

Теперь находим y из Ly = b, при помощи встроенной функции np.zeros_like(b), которая возвращает нулевую матрицу с той же размерностью, что и у вектора b.

In [74]:
def get_y(l, b):   
    rows = l.shape[0]  
    y = np.zeros_like(b, dtype=np.double);   
    y[0] = b[0] / l[0, 0]    
    for i in range(1, rows):
        y[i] = (b[i] - np.dot(l[i,:i], y[:i])) / l[i,i]        
    return y

А затем из уравнения Ux = y мы находим x:

In [75]:
def get_x(u, y):
    rows = u.shape[0]
    x = np.zeros_like(y, dtype=np.double);
    x[-1] = y[-1] / u[-1, -1]
    for i in range(rows-2, -1, -1):
        x[i] = (y[i] - np.dot(u[i,i:], x[i:])) / u[i,i]   
    return x

Функция решения записывается в 3 строчки:

In [76]:
def solve_lu(a, b):  
    l, u = a_to_lu(a) 
    y = get_y(l, b)
    return get_x(u, y)

Протестируем наш код на простом примере (с матрицей от 1 до 9 возникали трудности, поэтому изменил последний член на 10):

In [90]:
a = np.array([[1., 2., 3.], [4., 5., 6.], [7., 8., 10.]])
b = np.array([[1], [0], [0]])

solve_lu(a, b)

array([[-0.66666667],
       [-0.66666667],
       [ 1.        ]])

In [80]:
%timeit solve_lu(a,b)

47.6 µs ± 922 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Сравним ответ с linalg.solve, который реализован по тому же алгоритму в numpy:

In [78]:
np.linalg.solve(a, b)

array([[-0.66666667],
       [-0.66666667],
       [ 1.        ]])

Протестируем наш код на более сложном примере, найденном на просторах интернета:

In [87]:
c = np.array([[1, 4, 5], [6, 8, 22], [32, 5., 5]])
d = np.array([1, 2, 3.])

solve_lu(c, d)

array([ 0.05614973,  0.25935829, -0.01871658])

In [88]:
%timeit lu_solve(c,d)

37.6 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Снова сравним со встроенной функцией ответ:

In [89]:
np.linalg.solve(c, d)

array([ 0.05614973,  0.25935829, -0.01871658])

## PLU-алгоритм
Для реализации этого алгоритма будем использовать те же функции get_y, get_x, но изменил функцию solve и напишем новую функцию для получения самих матриц P,L,U. Для получения матрицы P мне все же пришлось использовать двойной цикл for, но он идет не каждый раз по всем строкам, что заметно ускоряет процесс. Используем также функцию ~np.isclose(), которая в случае максимального приближения значения к нулю завершит внутренний цикл.

In [85]:
def a_to_plu(a):
    rows = a.shape[0]
    u = a.copy()
    l = np.eye(rows, dtype=np.double)
    p = np.eye(rows, dtype=np.double)
    for i in range(rows):
        for j in range(i, rows):
            if ~np.isclose(u[i, i], 0.0):
                break
            u[[j, j+1]] = u[[j+1, j]]
            p[[j, j+1]] = p[[j+1, j]]
        f = u[i+1:, i] / u[i, i]
        l[i+1:, i] = f
        u[i+1:] -= f[:, np.newaxis] * u[i]   
    return p,l,u

Функция решения поменяется лишь в одной строчке:

In [86]:
def solve_plu(a, b):
    p,l,u = a_to_plu(a)
    y = get_y(l, np.dot(p, b))
    return get_x(u, y)

Протестируем наш код на простом примере:

In [92]:
solve_plu(a, b)

array([[-0.66666667],
       [-0.66666667],
       [ 1.        ]])

In [94]:
%timeit solve_plu(a,b)

137 µs ± 1.95 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Сверяем со встроенной функцией:

In [93]:
np.linalg.solve(a, b)

array([[-0.66666667],
       [-0.66666667],
       [ 1.        ]])

Проведём второй тест:

In [95]:
solve_plu(c,d)

array([ 0.05614973,  0.25935829, -0.01871658])

In [96]:
%timeit solve_plu(c,d)

130 µs ± 737 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Сравним с ответом:

In [97]:
np.linalg.solve(c,d)

array([ 0.05614973,  0.25935829, -0.01871658])

# Вывод:
В матрице 3х3 из реализованных нами методов наиболее эффективным можно назвать метод LU, но метод PLU также достаточно эффективен и работает не так уж и медленно.